## Table of contents:
0. [Imports & fix random seed](#0)
1. [Load data](#1)
2. [Pre-processing](#2)
3. [Modelling](#3)
4. [Post-processing](#4)
5. [Create & dump submission file](#5)

# 0 - Imports & fix random seed <a name=0></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random

SEED = 1337

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

SAMPLE = False

/home/manomano/.pyenv/versions/3.7.0/envs/im_challenge/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# 1 - Load data <a name=1></a>

In [2]:
df_orders = pd.read_csv("data/ventes_2018.csv")
df_orders.columns = [x.lower() for x in df_orders.columns]

df_nom = pd.read_csv("data/nomenclature_produits.csv")
df_nom.columns = [x.lower() for x in df_nom.columns]

df_pdv = pd.read_csv("data/points_de_vente.csv")
df_pdv.columns = [x.lower() for x in df_pdv.columns]

In [3]:
if SAMPLE:
    df_orders = df_orders[df_orders.id_pdv == 124].copy()

# 2 - Pre-processing <a name=2></a>

### Create df_train & df_test (1 row by date X id_pdv X id_artc)

In [4]:
df_train = pd.DataFrame()
df_test = pd.DataFrame()

df_train["date"] = pd.date_range("2018-01-01", "2018-12-31")
df_test["date"] = pd.date_range("2019-01-01", "2019-03-31")

# create key so we can cross join dates with pdv & artc datasets
df_train["key"] = 1
df_test["key"] = 1
df_orders["key"] = 1

df_unique_pdv = df_orders[["key", "id_pdv"]].drop_duplicates()
df_unique_artc = df_orders[["key", "id_artc"]].drop_duplicates()

df_train = pd.merge(df_train, df_unique_pdv, on="key")
df_train = pd.merge(df_train, df_unique_artc, on="key")

df_test = pd.merge(df_test, df_unique_pdv, on="key")
df_test = pd.merge(df_test, df_unique_artc, on="key")

del df_train["key"]
del df_test["key"]
del df_orders["key"]

In [5]:
# add orders to df_train
df_orders["date"] = pd.to_datetime(df_orders["date"])

df_train = pd.merge(df_train,
                    df_orders,
                    on=["date", "id_pdv", "id_artc"],
                    how="left")

df_train["qte"] = df_train["qte"].fillna(0).map(int)

In [6]:
# calculate last-7-days orders and remove empty last-7-days orders
# note: this cell takes several minutes to execute
df_train = df_train.sort_values("date", ascending=True)
df_train["qte_7_days"] = df_train.groupby(["id_pdv", "id_artc"])["qte"].transform(lambda x: x.rolling(7, min_periods=1).sum())
df_train = df_train[df_train["qte_7_days"] > 0].copy()
del df_train["qte_7_days"]

### Create temporal features

In [7]:
df_train["weekday"] = df_train["date"].dt.weekday
df_test["weekday"] = df_test["date"].dt.weekday

df_train["week"] = df_train["date"].dt.week
df_test["week"] = df_test["date"].dt.week

/home/manomano/.pyenv/versions/3.7.0/envs/im_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  after removing the cwd from sys.path.
/home/manomano/.pyenv/versions/3.7.0/envs/im_challenge/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


### Redefine week feature

In [8]:
# redefine week as the number of weeks since 01.01.2018
df_test["week"] += 52

# with this definition, the last day of 2018 should be considered as week 53
df_train.loc[df_train["date"] == "2018-12-31", "week"] = 53

### Create log target

In [9]:
df_train["y"] = np.log1p(df_train["qte"])
df_test["y"] = 0

### Clean training dataset

In [10]:
def clean_train_dataset(df):
    """
    After manual investigation, I noticed that:
    - some stores were bugged on some days (missing or abnormal quantity sold)
    - public holidays (1st of May, 8th of May, etc.) were disrupting the model
    I decided to either remove some rows or "fix" somes rows using W+1 and W-1 data
    """

    # id_pdv = 109 => remove bugged days from 06.05.2018 to 12.06.2018
    df = df[(df["id_pdv"] != 109) | ((df.date <= "2018-05-06") | (df.date >= "2018-06-11"))].copy()

    # id_pdv = 115 => remove bugged days from 06.05.2018 to 12.06.2018
    df = df[(df["id_pdv"] != 115) | (df.date >= "2018-01-29")].copy()

    # calculate W+1 and W-1 target to fix public holidays and bugged days
    df["week_p1"] = df["week"] + 1
    df["week_m1"] = df["week"] - 1
    df = pd.merge(df,
                  df[["week_p1", "weekday", "id_pdv", "id_artc", "y"]].rename(columns={"week_p1":"week",
                                                                                       "y": "y_m1"}),
                  how="left",
                  on=["week", "weekday", "id_pdv", "id_artc"])
    df = pd.merge(df,
                  df[["week_m1", "weekday", "id_pdv", "id_artc", "y"]].rename(columns={"week_m1":"week",
                                                                                       "y": "y_p1"}),
                  how="left",
                  on=["week", "weekday", "id_pdv", "id_artc"])

    # fix bugged days
    df = _fix_pdv_dates(df, list_id_pdv=[2, 5, 51, 52, 53, 55], list_dates=["2018-03-30"])
    df = _fix_pdv_dates(df, list_id_pdv=[25], list_dates=["2018-12-08"])
    df = _fix_pdv_dates(df, list_id_pdv=[50], list_dates=["2018-01-14", "2018-09-16", "2018-11-25"])
    df = _fix_pdv_dates(df, list_id_pdv=[59], list_dates=["2018-06-04"])
    df = _fix_pdv_dates(df, list_id_pdv=[69, 86], list_dates=["2018-11-17", "2018-11-18"], wp1_only=True) # use W+1 only since W-1 is public holidays
    df = _fix_pdv_dates(df, list_id_pdv=[100], list_dates=["2018-03-01", "2018-03-02"])
    df = _fix_pdv_dates(df, list_id_pdv=[109], list_dates=["2018-10-14", "2018-10-15", "2018-10-16"])
    df = _fix_pdv_dates(df, list_id_pdv=[135], list_dates=["2018-09-03"])
    df = _fix_pdv_dates(df, list_id_pdv=[140], list_dates=["2018-02-28"])
    

    # fix public holidays
    for d in ["2018-04-02", "2018-05-10", "2018-05-21", "2018-07-14", "2018-08-15", "2018-11-01", "2018-11-11"]:
        df.loc[df["date"] == d, "y"] = (0.5*df.loc[df["date"] == d, "y_m1"] +
                                        0.5*df.loc[df["date"] == d, "y_p1"]) 

    for d in ["2018-04-30", "2018-05-01", "2018-05-02"]:
        df.loc[df["date"] == d, "y"] = df.loc[df["date"] == d, "y_m1"]

    for d in ["2018-05-07", "2018-05-08", "2018-05-09"]:
        df.loc[df["date"] == d, "y"] = df.loc[df["date"] == d, "y_p1"]
        
    return df

def _fix_pdv_dates(df, list_id_pdv, list_dates, wp1_only=False):
    """
    For each pdv X date: fix the target using W-1 and W+1 data
    If wp1_only is set to True, only use W+1 (ignore W-1)
    """
    for id_pdv in list_id_pdv:
        for date in list_dates:
            filtre = (df["id_pdv"] == id_pdv) & (df["date"] == date)
            if wp1_only:
                df.loc[filtre , "y"] = df.loc[filtre, "y_p1"]
            else:
                df.loc[filtre , "y"] = (0.5*df.loc[filtre, "y_m1"] +
                                        0.5*df.loc[filtre, "y_p1"]) 

    return df

In [11]:
df_train = clean_train_dataset(df_train)

### Concatenate df_train and df_test

In [12]:
df_full = pd.concat((df_train, df_test), axis=0, sort=False)

### Change granularity to week X id_pdv X id_artc

In [13]:
df_full_week = pd.pivot_table(df_full,
                              index=["id_pdv", "id_artc", "week"],
                              columns="weekday",
                              values="y")

df_full_week.columns = [f"y{x}" for x in df_full_week.columns]
df_full_week = df_full_week.reset_index()

In [14]:
# save target columns
columns_target = [col for col in df_full_week.columns if col.startswith("y")]

In [15]:
# fill NAs target
df_full_week = df_full_week.fillna(0)

### Add pdv & artc metadata

In [16]:
df_full_week = pd.merge(df_full_week, df_nom, on="id_artc")
df_full_week = pd.merge(df_full_week, df_pdv, on="id_pdv")

### Add target at granularity week X weekday X id_pdv

In [17]:
df_pdv = df_full_week.groupby(["week", "id_pdv"])[columns_target].transform(np.mean)
df_pdv.columns = [x+"_pdv" for x in df_pdv.columns]
df_full_week = pd.concat((df_full_week, df_pdv), axis=1)

### Add target at granularity week X weekday X id_artc

In [18]:
df_artc = df_full_week.groupby(["week", "id_artc"])[columns_target].transform(np.mean)
df_artc.columns = [x+"_artc" for x in df_artc.columns]
df_full_week = pd.concat((df_full_week, df_artc), axis=1)

### Create quarter feature

In [19]:
# Note: since the dataset is at week granularity, we can't use the dt.quarter function
df_full_week["real_week"] = df_full_week["week"]
df_full_week.loc[df_full_week["real_week"] >= 53, "real_week"] -= 52
df_full_week["quarter"] = 4
df_full_week.loc[df_full_week.real_week <= 39, 'quarter'] = 3
df_full_week.loc[df_full_week.real_week <= 26, 'quarter'] = 2
df_full_week.loc[df_full_week.real_week <= 13, 'quarter'] = 1
del df_full_week["real_week"]

# 3 - Modelling <a name=3></a>

In [20]:
from lightgbm import LGBMRegressor
import keras

from sklearn.metrics import mean_squared_error
from keras.wrappers.scikit_learn import KerasRegressor

def _get_te_feature_mapping(df_week, group):
    """
    Get the mapping dataframe between the group columns and the target columns
    (that can be joined to the main dataframe)
    """
    if "weekday" in group:
        group_wout_weekday = [c for c in group if c != "weekday"]
        df_mapping = df_week.groupby(group_wout_weekday)[columns_target].mean()
        df_mapping.columns = ["x_te_"+"_".join(group_wout_weekday)+"_"+x for x in df_mapping.columns]
        df_mapping = df_mapping.reset_index()
    else:
        df_mapping = df_week.groupby(group)[columns_target].mean().mean(axis=1)
        df_mapping = df_mapping.reset_index()
        df_mapping.columns = group + ["x_te_"+'_'.join(group)]
        
    return df_mapping

def add_te_features(X, df, group, min_week, max_week, weeks_to_ignore):
    """
    Target encode within group columns. Features are:
    - calculated between min_week and max_week
    - calculated ignoring weeks_to_ignore weeks
    - concatenated to X dataframe
    """
    df_mapping = _get_te_feature_mapping(df[(df["week"] <= max_week) &
                                           (df["week"] >= min_week) & 
                                           (~df["week"].isin(weeks_to_ignore))], group)
    
    list_col_merge = [x for x in group if x != "weekday"]
    
    X = pd.merge(X,
                 df_mapping,
                 on=list_col_merge,
                 how="left")
    
    return X

Using TensorFlow backend.


In [21]:
# number of weeks used to calculate historical target values (at different granularities)
NB_WEEKS_HISTORIC = 6

# number of weeks we want to ignore in december
LAG = 1 

# lists that will contain prediction dataframes
list_df_preds_nn = []
list_df_preds_gbm = []

def nn_model():
    model = keras.models.Sequential()
    
    model.add(keras.layers.Dense(256, activation='relu', input_dim=input_dims))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(output_dims, activation='linear'))
    
    model.compile(loss="mse",
                  optimizer=keras.optimizers.Adam(lr=0.00005),
                  metrics=[[keras.metrics.RootMeanSquaredError(name='rmse')]])
    
    return model


# iterate over models (nn and gbm) and weeks
# note: since the predictions are blended at the end and are the same for every week, 
# we only iterate over 5 weeks to make the program run faster
for MODEL in ["nn", "gbm"]:
    for WEEK_PREDICT in [1, 4, 7, 10, 13]:
        
        print(f"MODEL:{MODEL} - WEEK_PREDICT:{WEEK_PREDICT}")

        # remove rows on which we can't have full historic
        X = df_full_week[df_full_week["week"] >= LAG + WEEK_PREDICT + NB_WEEKS_HISTORIC].copy()

        # historical target values at different granularities
        X["week_saved"] = X["week"]
        for nb_weeks_lag in range(LAG+WEEK_PREDICT, LAG+WEEK_PREDICT+NB_WEEKS_HISTORIC):
            X["week"] = X["week_saved"] - nb_weeks_lag

            X = pd.merge(X,
                         df_full_week[["id_pdv", "id_artc", "week"]+[c for c in df_full_week.columns if c.startswith("y")]],
                         on=["id_pdv", "id_artc", "week"],
                         how="left",
                         suffixes=("", f"_{nb_weeks_lag}_x"))

            # don't use the last 3 days of W51 (this was tuned locally)
            if nb_weeks_lag == (LAG+WEEK_PREDICT):
                to_keep = [c for c in X.columns if c not in [f'y4_{nb_weeks_lag}_x',
                                                             f'y5_{nb_weeks_lag}_x',
                                                             f'y6_{nb_weeks_lag}_x',
                                                             f'y4_pdv_{nb_weeks_lag}_x',
                                                             f'y5_pdv_{nb_weeks_lag}_x',
                                                             f'y6_pdv_{nb_weeks_lag}_x',
                                                             f'y4_artc_{nb_weeks_lag}_x',
                                                             f'y5_artc_{nb_weeks_lag}_x',
                                                             f'y6_artc_{nb_weeks_lag}_x']]
                X = X[to_keep]

        X["week"] = X["week_saved"]
        del X["week_saved"]

        # add target encoding features   
        for group in [["weekday", "lb_vent_faml"],
                      ["weekday", "id_artc"],
                      ["weekday", "id_pdv"],
                      ["weekday", "id_voct"],
                      ["weekday", "id_artc", "id_pdv"],
                      ["weekday", "id_voct", "nb_cais_grp"],
                      ['weekday', 'id_pdv', 'lb_vent_faml'],
                      ['weekday', 'lb_vent_faml', 'nb_cais_grp'],
                      ["lb_vent_faml", "id_regn", "nb_cais_grp"],
                      ["weekday", "id_voct", "id_artc"],
                      ["weekday", "id_regn", "id_artc"],
                      ['weekday', 'lb_vent_sous_faml'],
                      ['weekday', 'quarter', "id_pdv"],
                      ['weekday', 'quarter', "lb_vent_faml"]]:
            
            # ignore some weeks to calculate target encoding features
            if MODEL == "nn":
                weeks_to_ignore = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                                   16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
                                   29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 51, 52]
            elif MODEL == "gbm":
                weeks_to_ignore = [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
                                   25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
                                   37, 38, 51, 52]

            X = add_te_features(X,
                                df_full_week,
                                group,
                                min_week=0,
                                max_week=50,
                                weeks_to_ignore=weeks_to_ignore)

        # fill NAs added by target encoding features
        X = X.fillna(0)

        # train/test split
        X_train = X[X["week"] <= 50]
        X_test = X[X["week"] == (52 + WEEK_PREDICT)].copy()  
        y_train = X_train[columns_target]

        # create list of features
        features = [c for c in X_train.columns if c.endswith("_x") or c.startswith("x_te_")]
        
        print(X_train.shape, X_test.shape, len(features))

        if MODEL == "nn":
            df_preds = X_test[["id_pdv", "id_artc", "week"]].copy()
            
            input_dims = len(features)
            output_dims = 7

            # initiate nn
            nn = KerasRegressor(build_fn=nn_model)
            
            # fit
            history = nn.fit(
                X_train[features].values, y_train.values, 
                epochs=10,
                batch_size=1000,
                verbose=0,
                shuffle=True  
            )

            # predict
            preds = nn.predict(X_test[features])
            
            for i in range(7):
                df_preds[i] = preds[:, i]

            list_df_preds_nn.append(df_preds)

        elif MODEL == "gbm":
            df_preds = X_test[["id_pdv", "id_artc", "week"]].copy()

            # 1 model by weekday
            for i in range(7):
                target = "y"+str(i)
                
                # initiate lgb
                lgb = LGBMRegressor(nthread=16,
                                    objective='rmse',
                                    n_estimators=100,
                                    num_leaves=21,
                                    learning_rate=0.05,
                                    subsample=0.8,
                                    subsample_freq=1,
                                    colsample_bytree=0.3,
                                    min_child_samples=30,
                                    random_state=SEED)
                
                # fit
                lgb.fit(X_train[features],
                        y_train[target])
                
                # predict
                df_preds[i] = lgb.predict(X_test[features])

            list_df_preds_gbm.append(df_preds)

MODEL:nn - WEEK_PREDICT:1
(52713, 241) (1831, 241) 209
MODEL:nn - WEEK_PREDICT:4
(49202, 241) (1831, 241) 209
MODEL:nn - WEEK_PREDICT:7
(45645, 241) (1831, 241) 209
MODEL:nn - WEEK_PREDICT:10
(42024, 241) (1831, 241) 209
MODEL:nn - WEEK_PREDICT:13
(38418, 241) (1831, 241) 209
MODEL:gbm - WEEK_PREDICT:1
(52713, 241) (1831, 241) 209
MODEL:gbm - WEEK_PREDICT:4
(49202, 241) (1831, 241) 209
MODEL:gbm - WEEK_PREDICT:7
(45645, 241) (1831, 241) 209
MODEL:gbm - WEEK_PREDICT:10
(42024, 241) (1831, 241) 209
MODEL:gbm - WEEK_PREDICT:13
(38418, 241) (1831, 241) 209


# 4 - Post-processing <a name=4></a>

### Calculate average predictions by weekday

In [22]:
df_preds_gbm = pd.concat(list_df_preds_gbm, axis=0)
df_preds_nn = pd.concat(list_df_preds_nn, axis=0)

df_preds_gbm = df_preds_gbm.melt(id_vars=["id_pdv", "id_artc", "week"], var_name=["weekday"], value_name="pred")
df_preds_nn = df_preds_nn.melt(id_vars=["id_pdv", "id_artc", "week"], var_name=["weekday"], value_name="pred")

df_preds_gbm = df_preds_gbm.groupby(["id_pdv", "id_artc", "weekday"])["pred"].mean().reset_index()
df_preds_nn = df_preds_nn.groupby(["id_pdv", "id_artc", "weekday"])["pred"].mean().reset_index()

df_preds_gbm = pd.merge(df_test, df_preds_gbm, on=["id_pdv", "id_artc", "weekday"], how="left")
df_preds_nn = pd.merge(df_test, df_preds_nn, on=["id_pdv", "id_artc", "weekday"], how="left")

### Averaging NN and GBM predictions

In [23]:
df_preds = pd.merge(df_preds_gbm, df_preds_nn, on=["date", "id_pdv", "id_artc"])

df_preds["pred"] = 0.5*df_preds["pred_x"] + 0.5*df_preds["pred_y"]

### Calculate the final predictions

In [24]:
df_preds["qte"] = np.exp(df_preds["pred"]) - 1

In [25]:
df_preds.loc[df_preds["qte"] < 0, "qte"] = 0

### Cast to int (choose between floor and ceil by minimizing MSE)

In [26]:
# calculate squared error for floor & ceil casts
df_preds["mse_floor"] = df_preds["qte"].map(lambda x: (np.log1p(x) - np.log1p(np.floor(x)))**2)
df_preds["mse_ceil"] = df_preds["qte"].map(lambda x: (np.log1p(x) - np.log1p(np.ceil(x)))**2)

# choose between the cast with the lowest squared error
filtre = df_preds["mse_floor"] < df_preds["mse_ceil"]
df_preds.loc[filtre, "qte"] = np.floor(df_preds.loc[filtre, "qte"])
df_preds.loc[~filtre, "qte"] = np.ceil(df_preds.loc[~filtre, "qte"])

# cast dtype to int
df_preds["qte"] = df_preds["qte"].map(int)

### Post processing prediction

In [27]:
# this list was created manually
# I suppose these stores are usually closed on sunday by looking at their historical data
# this could be industrialized knowing the stores opening schedule
id_pdv_not_opened_on_sunday = [12, 128, 149, 110, 102, 14, 75,
                               139, 38, 127, 4, 33, 73, 130, 52,
                               98, 129, 140, 96, 87, 5, 51,
                               55, 53, 146, 2, 65, 89]

df_preds["date"] = pd.to_datetime(df_preds["date"])
df_preds["weekday"] = df_preds["date"].dt.weekday

# force "supposed closed pdv on sunday" predictions to 0
df_preds.loc[(df_preds["id_pdv"].isin(id_pdv_not_opened_on_sunday)) & (df_preds["weekday"] == 6), "qte"] = 0

# force 1st of January predictions to 0
df_preds.loc[df_preds["date"] == "2019-01-01", "qte"] = 0

# keep only rows where qte > 0
df_preds = df_preds[df_preds["qte"] > 0].copy()

# 5 - Create & dump submission file <a name=5></a>

In [28]:
ID_SUB = 1

In [29]:
df_preds["id"] = (df_preds["id_pdv"].map(str) + "_"
                  + df_preds["id_artc"].map(str) + "_"
                  + df_preds["date"].dt.strftime('%Y%m%d'))

df_preds[["id", "qte"]].to_csv(f"submits/{ID_SUB}.csv", index=False)

!zip submits/{ID_SUB}.zip submits/{ID_SUB}.csv

updating: submits/1.csv (deflated 85%)
